In [ ]:
from tools import analysistools as atools
from functools import partial as partial
from ga import networkedgeneticalgorithm as nga
from db import databaseconnection as dbconn
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0)

In [ ]:
epsIndices = range(4, 5)
epsDBs = ['/mnt/c/Users/manish/Documents/GitHub/optihedron/db/GP{}.db'.format(i) for i in epsIndices]
print(epsDBs[0])

In [ ]:
dropIndProps = partial(atools.dropChildren, parentKey = 'individuals', childKeys = ['genome', 'gh'])
dropGens = partial(atools.dropParentsConditional, parentKey = 'individuals', condition = lambda ind: ind['gen'] <= 30)
dropMetrics = partial(atools.dropParents, parentKey='metrics')
dropGenealogy = partial(atools.dropParents, parentKey='genealogy')

In [ ]:
dropTasks = [dropMetrics, dropGenealogy, dropIndProps, dropGens]
cleanTasks = [atools.cleanLigands, atools.cleanFits]
sortTasks = [atools.sortIndGens]

epsData = atools.load(epsDBs, dbconn, clean = cleanTasks, drop = dropTasks, sort = sortTasks)

# PLOTS

In [ ]:
epsLabel = 'ligand strength'
purpleMap = sns.cubehelix_palette(as_cmap=True)
BlueRedMap = 'RdBu_r'

## phase plots

In [ ]:
epsLabel = 'ligand strength'
purpleMap = sns.cubehelix_palette(as_cmap=True)
BlueRedMap = 'RdBu_r'

In [ ]:
sortedData = []
for epsDatum in epsData:
    sortedData.append(sortIndGens(epsDatum))
epsData = sortedData
plotScan = partial(atools.plotScanGen, scanData=epsData, scanLabel=epsLabel, scanIndices=epsIndices, indexOffset=epsIndices[0], annotate=True)

In [ ]:
# sortedData = []
# for epsDatum in epsData:
#     sortedData.append(sortIndFits(epsDatum))
# epsData = sortedData
# plotScan = partial(atools.plotScanGen, scanData=epsData, scanLabel=epsLabel, scanIndices=epsIndices, indexOffset=epsIndices[0], annotate=True)

In [ ]:
# sortedData = []
# for epsDatum in epsData:
#     sortedData.append(sortIndGens(epsDatum))
#     sortedData.append(sortIndFits(epsDatum))
#     sortedData.append(sortIndLigandCount(epsDatum))
# epsData = sortedData
# plotScan = partial(atools.plotScanGen, scanData=epsData, scanLabel=epsLabel, scanIndices=epsIndices, indexOffset=epsIndices[0], annotate=True)

In [ ]:
#plotScan(interest=atools.characterScore, aggregateMode='AVG', plotName='character score - avg', cmap=BlueRedMap, vmin=-1, vmax=1)
#plotScan(interest=atools.characterScore, aggregateMode='MAX', plotName='character score - max', cmap=BlueRedMap, vmin=-1, vmax=1)
#plotScan(interest=atools.characterScore, aggregateMode='MIN', plotName='character score - min', cmap=BlueRedMap, vmin=-1, vmax=1)

#plotScan(interest=atools.lineyABS, aggregateMode='AVG', plotName='number of liney ligands - avg', cmap=purpleMap)
#plotScan(interest=atools.lineyABS, aggregateMode='MAX', plotName='number of liney ligands - max', cmap=purpleMap)
#plotScan(interest=atools.lineyABS, aggregateMode='MIN', plotName='number of liney ligands - min', cmap=purpleMap)

#plotScan(interest=atools.lineyREL, aggregateMode='AVG', plotName='fraction of liney ligands - avg', cmap=purpleMap)
#plotScan(interest=atools.lineyREL, aggregateMode='MAX', plotName='fraction of liney ligands - max', cmap=purpleMap)
#plotScan(interest=atools.lineyREL, aggregateMode='MIN', plotName='fraction of liney ligands - min', cmap=purpleMap)

#plotScan(interest=atools.spottyABS, aggregateMode='AVG', plotName='number of spotty ligands - avg', cmap=purpleMap)
#plotScan(interest=atools.spottyABS, aggregateMode='MAX', plotName='number of spotty ligands - max', cmap=purpleMap)
#plotScan(interest=atools.spottyABS, aggregateMode='MIN', plotName='number of spotty ligands - min', cmap=purpleMap)

#plotScan(interest=atools.spottyREL, aggregateMode='AVG', plotName='fraction of spotty ligands - avg', cmap=purpleMap)
#plotScan(interest=atools.spottyREL, aggregateMode='MAX', plotName='fraction of spotty ligands - max', cmap=purpleMap)
#plotScan(interest=atools.spottyREL, aggregateMode='MIN', plotName='fraction of spotty ligands - min', cmap=purpleMap)

#plotScan(interest=atools.patchyABS, aggregateMode='AVG', plotName='number of patchy ligands - avg', cmap=purpleMap)
#plotScan(interest=atools.patchyABS, aggregateMode='MAX', plotName='number of patchy ligands - max', cmap=purpleMap)
#plotScan(interest=atools.patchyABS, aggregateMode='MIN', plotName='number of patchy ligands - min', cmap=purpleMap)

#plotScan(interest=atools.patchyREL, aggregateMode='AVG', plotName='fraction of patchy ligands - avg', cmap=purpleMap)
#plotScan(interest=atools.patchyREL, aggregateMode='MAX', plotName='fraction of patchy ligands - max', cmap=purpleMap)
#plotScan(interest=atools.patchyREL, aggregateMode='MIN', plotName='fraction of patchy ligands - min', cmap=purpleMap)

#plotScan(interest=lambda ind: float(ind['fitness']), aggregateMode='AVG', plotName='fitness - avg', fmt='.2f', cmap=purpleMap)
#plotScan(interest=lambda ind: float(ind['fitness']), aggregateMode='MAX', plotName='fitness - max', fmt='.2f', cmap=purpleMap)
#plotScan(interest=lambda ind: float(ind['fitness']), aggregateMode='MIN', plotName='fitness - min', fmt='.2f', cmap=purpleMap)

#plotScan(interest=lambda ind: float(len(ind['phenome'].particle.ligands)), aggregateMode='AVG', plotName='number of ligands - avg', cmap=purpleMap)
#plotScan(interest=lambda ind: float(len(ind['phenome'].particle.ligands)), aggregateMode='MAX', plotName='number of ligands - max', cmap=purpleMap)
#plotScan(interest=lambda ind: float(len(ind['phenome'].particle.ligands)), aggregateMode='MIN', plotName='number of ligands - min', cmap=purpleMap)

#plotScan(interest=lambda ind: float(np.sum([lig.eps for lig in ind['phenome'].particle.ligands])), aggregateMode='AVG', plotName='total ligand affinity - avg', fmt='.2f', cmap=purpleMap)
#plotScan(interest=lambda ind: float(np.sum([lig.eps for lig in ind['phenome'].particle.ligands])), aggregateMode='MAX', plotName='total ligand affinity - max', fmt='.2f', cmap=purpleMap)
#plotScan(interest=lambda ind: float(np.sum([lig.eps for lig in ind['phenome'].particle.ligands])), aggregateMode='MIN', plotName='total ligand affinity - min', fmt='.2f', cmap=purpleMap)

#plotScan(interest=atools.NNCountAVG, aggregateMode='AVG', plotName='average number of NNs - avg', cmap=purpleMap)
#plotScan(interest=atools.NNCountAVG, aggregateMode='MAX', plotName='average number of NNs - max', cmap=purpleMap)
#plotScan(interest=atools.NNCountAVG, aggregateMode='MIN', plotName='average number of NNs - min', cmap=purpleMap)

#plotScan(interest=atools.lineyChainCount, aggregateMode='AVG', plotName='number of liney chains - avg', cmap=purpleMap)
#plotScan(interest=atools.lineyChainCount, aggregateMode='MAX', plotName='number of liney chains - max', cmap=purpleMap)
#plotScan(interest=atools.lineyChainCount, aggregateMode='MIN', plotName='number of liney chains - min', cmap=purpleMap)

#plotScan(interest=atools.lineyChainSizeAVG, aggregateMode='AVG', plotName='average size of liney chains - avg', cmap=purpleMap)
#plotScan(interest=atools.lineyChainSizeAVG, aggregateMode='MAX', plotName='average size of liney chains - max', cmap=purpleMap)
#plotScan(interest=atools.lineyChainSizeAVG, aggregateMode='MIN', plotName='average size of liney chains - min', cmap=purpleMap)

#plotScan(interest=atools.lineyChainSizeMAX, aggregateMode='AVG', plotName='size of largest liney chain - avg', cmap=purpleMap)
#plotScan(interest=atools.lineyChainSizeMAX, aggregateMode='MAX', plotName='size of largest liney chain - max', cmap=purpleMap)
#plotScan(interest=atools.lineyChainSizeMAX, aggregateMode='MIN', plotName='size of largest liney chain - min', cmap=purpleMap)

#plotScan(interest=atools.lineyChainSizeMIN, aggregateMode='AVG', plotName='size of smallest liney chain - avg', cmap=purpleMap)
#plotScan(interest=atools.lineyChainSizeMIN, aggregateMode='MAX', plotName='size of smallest liney chain - max', cmap=purpleMap)
#plotScan(interest=atools.lineyChainSizeMIN, aggregateMode='MIN', plotName='size of smallest liney chain - min', cmap=purpleMap)